In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import warnings
import matplotlib.colors as colors
import matplotlib.animation as animation
import matplotlib.lines as lines
import datetime
import math
%matplotlib notebook

In [4]:
####------- Import data for demonstration
warnings.filterwarnings(action='once')

!pip install sociophysicsdatahandler
from sociophysicsDataHandler import SociophysicsDataHandler

!cat auth.txt

dh = SociophysicsDataHandler()
dh.list_files('ut')

def fetchData(path):
  #Fetch and combine all data from a directory
  file_list = dh.list_files(path)
  print(dh.filelist)
  n = file_list.name.size
  df_array = np.empty(n, dtype=object)

  for i in range(n):
    dh.fetch_prorail_data_from_path(path + "/" + file_list.name[i]);
    #Convert the date time column to actual date time
    dh.df['date_time_utc'] = pd.to_datetime(dh.df['date_time_utc'], unit = 'ms')
    #Swap x and y
    column_titles = {'date_time_utc':'date_time_utc', 'tracked_object':'tracked_object', 'x_pos':'y_pos', 'y_pos':'x_pos'}
    dh.df.rename(columns = column_titles, inplace = True)

    df_array[i] = dh.df

  return pd.concat(df_array)


file_path = 'ehv/platform2.1/20220404/'

specific_file = 'ehv/platform2.1/20220404/EHV_Platform2.1_2022040409_trajectorie.parquet' 

# fetch the data from the first file
#df = fetchData(file_path);

#These lines might be redundant
dh.fetch_prorail_data_from_path(specific_file)
dh.df['date_time_utc'] = pd.to_datetime(dh.df['date_time_utc'], unit = 'ms')
column_titles = {'date_time_utc':'date_time_utc', 'tracked_object':'tracked_object', 'x_pos':'y_pos', 'y_pos':'x_pos'}
dh.df.rename(columns = column_titles, inplace = True)

df = dh.df
dh.fetch_background_image_from_path('ehv/background_images/EHV.Perron2.1_multisensor.png')
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


guest
YDPGL-XREXC-QGMZS-UAKES
targeting path /storage/surfsara/ProRail_USE_LL_data/ut


C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


Files listed. Accessible as <this-object>.filelist
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404/EHV_Platform2.1_2022040409_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/background_images/EHV.Perron2.1_multisensor.png
background fetched. Accessible as <this-object>.bg


,date_time_utc,tracked_object,y_pos,x_pos
0,2022-04-04 07:00:00,4222520,-2102.0,62187.0
1,2022-04-04 07:00:00,4222553,2395.0,10207.0
2,2022-04-04 07:00:00,4222554,1519.0,24356.0
3,2022-04-04 07:00:00,4222590,-641.0,27013.0
4,2022-04-04 07:00:00,4222607,433.0,42440.0


In [13]:
####------- Labelling code
def above_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    above_line_series = df[y_tag] > - math.atan(alpha/360*2*math.pi)*(df[x_tag]-line[0])+line[1]
    return above_line_series

def below_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    below_line_series = df[y_tag] < - math.atan(alpha/360*2*math.pi)*(df[x_tag]-line[0])+line[1]
    return below_line_series

df_grouped_by_id = df.groupby('tracked_object')
df_object_start_location = df_grouped_by_id.first()
df_object_end_location = df_grouped_by_id.last()

alpha = 0.8 
boarding_line_1 = [-2e4,2e3] #x, y, angle (degrees)
dx = 5e4
boarding_line_1.append(boarding_line_1[0]+dx) # [2]: x of point 2
boarding_line_1.append(boarding_line_1[1]-dx*math.tan(alpha/360*2*math.pi)) # [3]: y of point 2

boarding_line_2 = [-2e4,-0.65e4] #x, y, angle (degrees)
dx = 5e4
boarding_line_2.append(boarding_line_2[0]+dx) # [2]: x of point 2
boarding_line_2.append(boarding_line_2[1]-dx*math.tan(alpha/360*2*math.pi)) # [3]: y of point 2

#vertical line
boarding_line_3 = [70000,-15000,70000,10000]

#If start location is above line 1, offboarding. same for below line 2. Also to the right of the vertical line
offboarding_1 = above_line(df_object_start_location,boarding_line_1,alpha)
offboarding_2 = below_line(df_object_start_location,boarding_line_2,alpha)
offboarding_3 = (df_object_start_location['x_pos']>boarding_line_3[0])
offboarding = offboarding_1 | offboarding_2 | offboarding_3

#If end location is above line 1, onboarding. same for below line 2.
onboarding_1 = above_line(df_object_end_location,boarding_line_1,alpha)
onboarding_2 = below_line(df_object_end_location,boarding_line_2,alpha)
onboarding_3 = (df_object_end_location['x_pos']>boarding_line_3[0])
onboarding = onboarding_1 | onboarding_2 | onboarding_3

df_object_start_location['offboarding'] = offboarding
df_object_end_location['onboarding'] = onboarding
offboarding.value_counts(), onboarding.value_counts()



df['onboarding_or_offboarding'] = 0
df['offboarding'] = df['tracked_object'].map(df_object_start_location['offboarding'])
df['onboarding'] = df['tracked_object'].map(df_object_end_location['onboarding'])
print(df['offboarding'].value_counts())
df.loc[df['onboarding'] == True, 'onboarding_or_offboarding'] = 1  #1: onboarding
df.loc[df['offboarding'] == True, 'onboarding_or_offboarding'] = 2 #2: offboarding
df.head()
series_onboarding_or_offboarding = df.groupby('tracked_object').first()['onboarding_or_offboarding']
series_onboarding_or_offboarding.head()

#series_onboarding_or_offboarding is now a series where you find if a certain id is onboarding of offboarding

list_onboarding = series_onboarding_or_offboarding.loc[series_onboarding_or_offboarding==1].index.values.tolist()
list_offboarding = series_onboarding_or_offboarding.loc[series_onboarding_or_offboarding==2].index.values.tolist()
)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


False    1193830
True      511395
Name: offboarding, dtype: int64
[4222520, 4222590, 4222615, 4222621, 4222877, 4222922, 4222970, 4222987, 4222991, 4222994, 4223006, 4223067, 4223074, 4223075, 4223084, 4223090, 4223096, 4223103, 4223105, 4223108, 4223111, 4223119, 4223141, 4223153, 4223158, 4223170, 4223172, 4223177, 4223181, 4223185, 4223186, 4223189, 4223192, 4223196, 4223197, 4223212, 4223220, 4223223, 4223240, 4223245, 4223248, 4223254, 4223261, 4223267, 4223269, 4223286, 4223288, 4223290, 4223295, 4223297, 4223298, 4223306, 4223307, 4223309, 4223315, 4223321, 4223327, 4223328, 4223329, 4223334, 4223335, 4223338, 4223348, 4223351, 4223355, 4223357, 4223359, 4223366, 4223369, 4223370, 4223372, 4223374, 4223376, 4223378, 4223379, 4223380, 4223381, 4223382, 4223397, 4223400, 4223401, 4223405, 4223413, 4223417, 4223422, 4223427, 4223428, 4223430, 4223432, 4223433, 4223435, 4223440, 4223448, 4223450, 4223463, 4223469, 4223470, 4223474, 4223483, 4223484, 4223487, 4223489, 4223490, 422349